In [1]:
import pennylane as qml
from pennylane import numpy as np
from pennylane.templates import RandomLayers
import pandas as pd

import time
import os
# os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2' 
import tensorflow as tf
from tensorflow import keras
import json

import matplotlib.pyplot as plt
from tqdm.notebook import tqdm

In [2]:
from qnn_utils import get_mnist_data, get_quantum_data, CNNModel

## Load Data

In [3]:
train_images, val_images, test_images, train_labels, val_labels, test_labels = get_mnist_data(n_train=2500, n_test=500)

## Learn Models

In [4]:
statistics = {"n_layers":[], "n_wires":[], "time_processing":[], "Score":[]}

In [5]:
%%time
# train Classic NN
c_model = CNNModel()
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=f"runs/ClassicModel", histogram_freq=1)
# train
c_history = c_model.fit(train_images, train_labels, validation_data=(val_images, val_labels),
                        batch_size=64, epochs=20, verbose=0, callbacks=[tensorboard_callback])
# validate
results = c_model.evaluate(test_images, test_labels, batch_size=64)
# update statistics
statistics["n_layers"].append("Classic")
statistics["n_wires"].append("Classic")
statistics["time_processing"].append("Classic")
statistics["Score"].append(results[1])

8/8 [==============================] - 0s 10ms/step - loss: 0.1519 - accuracy: 0.9640
Wall time: 38.2 s


In [6]:
%%time
for n_layers in range(1, 4):
    for n_wires in range(1, 5):        
        # get quantum data
        preprocess_time, q_train_images, q_val_images, q_test_images = get_quantum_data(train_images, val_images,
                                                                                        test_images, n_layers, n_wires)
        # train quantum NN
        q_model = CNNModel()
        tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=f"runs/QNNLayers_{n_layers}Wires_{n_wires}",
                                                              histogram_freq=1)
        q_history = q_model.fit(q_train_images, train_labels, validation_data=(q_val_images, val_labels),
                                batch_size=64, epochs=20, verbose=0, callbacks=[tensorboard_callback])
        # validate
        results = q_model.evaluate(q_test_images, test_labels, batch_size=64)
        # update statistics
        statistics["n_layers"].append(n_layers)
        statistics["n_wires"].append(n_wires)
        statistics["time_processing"].append(preprocess_time)
        statistics["Score"].append(results[1])
        # pirnt params
        print(f"Layers: {n_layers}, Wires: {n_wires}, Score: {results[1]}")

  0%|          | 0/2000 [00:00<?, ?it/s]

  0%|          | 0/500 [00:00<?, ?it/s]

  0%|          | 0/500 [00:00<?, ?it/s]

8/8 [==============================] - 0s 10ms/step - loss: 0.1405 - accuracy: 0.9640
Layers: 1, Wires: 1, Score: 0.9639999866485596


  0%|          | 0/2000 [00:00<?, ?it/s]

...

8/8 [==============================] - 0s 10ms/step - loss: 0.1406 - accuracy: 0.9660
Layers: 3, Wires: 3, Score: 0.9660000205039978


  0%|          | 0/2000 [00:00<?, ?it/s]

  0%|          | 0/500 [00:00<?, ?it/s]

  0%|          | 0/500 [00:00<?, ?it/s]

8/8 [==============================] - 0s 10ms/step - loss: 0.1086 - accuracy: 0.9660
Layers: 3, Wires: 4, Score: 0.9660000205039978
Wall time: 1d 9h 50min 18s


In [18]:
df = pd.DataFrame(statistics)
df = df.loc[df["n_wires"]!=1]
df

,n_layers,n_wires,time_processing,Score
0,Classic,Classic,Classic,0.964
2,1,2,6009.53169,0.960
3,1,3,8535.707037,0.968
4,1,4,11397.590142,0.976
6,2,2,8588.147336,0.960
7,2,3,11805.092293,0.976
8,2,4,16194.567904,0.970
10,3,2,11048.144514,0.972
11,3,3,14004.419777,0.966
12,3,4,20747.060564,0.966


## Dataset size

In [14]:
statistics = {"Model":[], "TrainSize":[], "time_processing":[], "Accuracy":[]}

In [5]:
# get quantum test
_, _, test_images, _, _, test_labels = get_mnist_data(n_train=10,
                                                                                              n_test=None)
preprocess_time, q_train_images, q_val_images, q_test_images = get_quantum_data(train_images, val_images,
                                                                                test_images, n_layers=1, n_wires=4)


  0%|          | 0/8 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/10000 [00:00<?, ?it/s]

In [15]:
for train_size in [100, 500, 1000, 5000, 20000, 30000]:
    # get data
    train_images, val_images, test_images, train_labels, val_labels, test_labels = get_mnist_data(n_train=train_size,
                                                                                                  n_test=None)
    
    # get quantum data
    preprocess_time, q_train_images, q_val_images = get_quantum_data(train_images, val_images,
                                                                                    test_images, 1, 4, return_test=False)
    
    # train Classic NN
    c_model = CNNModel()
    tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=f"runs/ClassicDataset_{train_size}", histogram_freq=1)
    # train
    c_history = c_model.fit(train_images, train_labels, validation_data=(val_images, val_labels),
                            batch_size=64, epochs=20, verbose=0, callbacks=[tensorboard_callback])
    # validate
    results = c_model.evaluate(test_images, test_labels, batch_size=64)
    # update statistics
    statistics["Model"].append("Classic")
    statistics["TrainSize"].append(train_size)
    statistics["time_processing"].append("Classic")
    statistics["Accuracy"].append(results[1])
    print(f"Classic, TrainSize: {train_size}, Score: {results[1]}")
    
    # train quantum NN
    q_model = CNNModel()
    tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=f"runs/QNNDataset_{train_size}",
                                                          histogram_freq=1)
    q_history = q_model.fit(q_train_images, train_labels, validation_data=(q_val_images, val_labels),
                            batch_size=64, epochs=20, verbose=0, callbacks=[tensorboard_callback])
    # validate
    results = q_model.evaluate(q_test_images, test_labels, batch_size=64)
    # update statistics
    statistics["Model"].append("QNN")
    statistics["TrainSize"].append(train_size)
    statistics["time_processing"].append(preprocess_time)
    statistics["Accuracy"].append(results[1])
    # pirnt params
    print(f"QNN, TrainSize: {train_size}, Score: {results[1]}")

  0%|          | 0/80 [00:00<?, ?it/s]

  0%|          | 0/20 [00:00<?, ?it/s]

157/157 [==============================] - 2s 10ms/step - loss: 1.2875 - accuracy: 0.7366
Classic, TrainSize: 100, Score: 0.7365999817848206
157/157 [==============================] - 2s 11ms/step - loss: 1.3528 - accuracy: 0.6795
QNN, TrainSize: 100, Score: 0.6794999837875366


  0%|          | 0/400 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

...

  0%|          | 0/4000 [00:00<?, ?it/s]

157/157 [==============================] - 2s 11ms/step - loss: 0.0619 - accuracy: 0.9866
Classic, TrainSize: 20000, Score: 0.9865999817848206
157/157 [==============================] - 2s 11ms/step - loss: 0.9609 - accuracy: 0.7110
QNN, TrainSize: 20000, Score: 0.7110000252723694


  0%|          | 0/24000 [00:00<?, ?it/s]

  0%|          | 0/6000 [00:00<?, ?it/s]

157/157 [==============================] - 2s 10ms/step - loss: 0.0525 - accuracy: 0.9897
Classic, TrainSize: 30000, Score: 0.9897000193595886
157/157 [==============================] - 2s 11ms/step - loss: 0.4856 - accuracy: 0.8621
QNN, TrainSize: 30000, Score: 0.8621000051498413


In [16]:
statistics

{'Model': ['Classic',
  'QNN',
  'Classic',
  'QNN',
  'Classic',
  'QNN',
  'Classic',
  'QNN',
  'Classic',
  'QNN',
  'Classic',
  'QNN'],
 'TrainSize': [100,
  100,
  500,
  500,
  1000,
  1000,
  5000,
  5000,
  20000,
  20000,
  30000,
  30000],
 'time_processing': ['Classic',
  375.2024953365326,
  'Classic',
  1858.2192106246948,
  'Classic',
  3726.7459485530853,
  'Classic',
  18708.304575681686,
  'Classic',
  74702.27434682846,
  'Classic',
  111936.38474965096],
 'Accuracy': [0.7365999817848206,
  0.6794999837875366,
  0.8892999887466431,
  0.8968999981880188,
  0.939300000667572,
  0.9236000180244446,
  0.9764999747276306,
  0.9732999801635742,
  0.9865999817848206,
  0.7110000252723694,
  0.9897000193595886,
  0.8621000051498413]}